For this program, you will be coding some of the ideas around Bayes classification.

**For most of this coding assignment, you may not use packages or "canned" code in your program other than simple function calls to needed math functions or file I/O. I.e., do not use a Bayes classifier that you did not write. However, for making histograms, plotting, or visualization, you may use other software. You may want to make this part of your package use.**

The input file is a csv file named *BayesAssign1_??.dat* and is located in the Data folder on Google. Note: ?? is a number. Look in the folder for possibly more than 1 data file. They were built with different parameters.

Each  line is an observation that has the class indicator, "NEG" or "POS", followed by numeric values for one feature. NEG indicates the absence of a disease or other characteristic. POS indicates the presence.

The values for each class come from Gaussian distributions. 

Split the data into train and test sets. Determine the statistics for each distribution (mean and variance), NEG and POS, from the training data. Using this information, classify the data in the test set.

Print to the console the following values:
* estimated mean and standard deviation for each class.
* estimated prior probability of each class
* percentage of the data used for training
* prevalence
* accuracy
* sensitivity
* specificity
* precision (positive predicitive value)
* a confusion matrix (does not have to be in matrix form - just label the numbers)

You may want to output your classified test data. If so, perhaps on each line would be: actual class, predicted class, value.

You could read this data into other programs for analysis, ...

**Other ideas**

* Make plots, on the same graph, of two Gaussians using the mean and variance you calculated.
* Plot histograms of the original data.
* How do your estimated parameters and performance vary with training fraction?
* Discuss how this might work if one or both of the distributions were uniform in the interval [a, b] instead of both Gaussian.
* Discuss how this might work for more than one class.
* Discuss how this might work for multi-dimensional (more than one feature) observations. E.g., 2-D Gaussians.






In [1]:
'''
Functions and imports for Bayes Assignment #1
'''
import math
import random
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

#Return the Gaussian probability density function for x given mean = m and sigma = s
def gauss_val(x, m, s):
  val = math.exp(-math.pow((x-m)/s,2)/2.0)/(s*math.sqrt(math.pi*2))
  return(val)

#Generate Gaussian values for each class
#  Add uniform noise
def gen_gauss_list(stats, cl_names, num, noise_fact):
  g_vals = []
  for i in range(num):
    c = random.random()
    #Is the sample from class 0 or from class 1?
    if (c < stats[2]):
      new_val = random.gauss(stats[0], stats[1])
      noise_val = random.uniform(-noise_fact*stats[1], noise_fact*stats[1])
      new_val += noise_val
      g_vals.append([cl_names[0], new_val])
    else:
      new_val = random.gauss(stats[3], stats[4])
      noise_val = random.uniform(-noise_fact*stats[4], noise_fact*stats[4])
      new_val += noise_val      
      g_vals.append([cl_names[1], new_val])
  return(g_vals)

In [2]:
'''
Generate data for Bayes Assignment #1. Assume 2 Gaussians for now

Students do not need this code, and it is only here in case of interest
  of if they want to make their own data to check their code.
  If so, change the output file name below.
Note: parameters given below are not those used for data provided.

'''

if __name__ == "__main__":

  #Class conditional parameters for two Gaussians.
  MEAN_0 = 0.0
  SIG_0  = 1.0
  PROB_0 = 0.5
  MEAN_1 = 1.0
  SIG_1  = 2.0
  PROB_1 = 1 - PROB_0

  STATS = [MEAN_0, SIG_0, PROB_0, MEAN_1, SIG_1, PROB_1]
  CLASS_NAMES = ["NEG", "POS"]

  SEED = 977894657
  random.seed(SEED)

  #Noise can be added as a fraction of the sigma for each class.
  NOISE_FACT = 0.0

  NUM_VALUES = 10000

  #Print parameters
  print("Actual parameters: {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}".format(MEAN_0, SIG_0, PROB_0, \
                                                                              MEAN_1, SIG_1, PROB_1))
  print("Noise factor: {:.4f}".format(NOISE_FACT))
  print()

  val_list = gen_gauss_list(STATS, CLASS_NAMES, NUM_VALUES, NOISE_FACT)

  #Output file name. Students should use their folder and name.
  out_file_name = 'MY OUTPUT FILE NAME'

  #Write output csv file
  with open(out_file_name, 'w') as out_file_ptr:
    for i in range(len(val_list)):
      pass
      out_str = "{:s}, {:7.4f}\n".format(val_list[i][0], val_list[i][1])
      out_file_ptr.write(out_str)


  print("END")


Actual parameters: 0.0000 1.0000 0.5000 1.0000 2.0000 0.5000
Noise factor: 0.0000

END


**Sample output from running  the data generation code.**

Actual parameters: 0.0000 1.0000 0.5000 1.0000 2.0000 0.5000

Noise factor: 0.0000

END


Imports

In [3]:
import math
import csv


In [4]:
class bayes:
  # splits data into positive and negate to calculate stats
  def split_classes(data):
    neg_data = []
    pos_data = []
    for i in range(len(data)):
      cl, num = data[i][0], float(data[i][1])
      if cl == 'NEG':
        neg_data.append([cl ,num])
      if cl == 'POS':
        pos_data.append([cl ,num])
    return neg_data, pos_data
 
  # calculates mean of a dataset
  def find_mean(data):
    sum = 0
    for i in range(len(data)):
      sum += data[i][1]
    m = sum / len(data)
    return m
  
  # calculates standard deviation or sigma of a dataset
  def find_std_dev(data,m):
    sq_sub_sum = 0
    for i in range(len(data)):
      sq_sub_sum += (data[i][1] - m)**2
    std_dev = math.sqrt(sq_sub_sum / len(data))
    return std_dev
  
  # finds how common an outcome is
  def find_prevalence(feat_data,all_data):
    prev = len(feat_data) / len(all_data)
    return prev

  # finds a favored splitpoint, more for checking  
  def split_point(d1_mean, d1_std_dev, d2_mean, d2_std_dev):
    sp = 1000
    diff = 1000
    for x in range(int(d1_mean), int(d2_mean*100), 1):
      x_it = x/100
      if abs(abs(x_it - d1_mean) / d1_std_dev - abs(x_it - d2_mean) / d2_std_dev) < diff:
        diff = abs(abs(x_it - d1_mean) / d1_std_dev - abs(x_it - d2_mean) / d2_std_dev)
        sp = x_it
    return sp
  
  # calculates gaussian value
  def gauss_val(x, m, s):
    val = math.exp(-math.pow((x-m)/s,2)/2.0)/(s*math.sqrt(math.pi*2))
    return(val)

  # makes predicitons on a class given gaussian values for positive and negative probabilities
  def find_predictions(data, neg_mean, neg_std_dev, pos_mean, pos_std_dev):
    predictions = []
    for i in range(len(data)):
      real_class, val = data[i][0], float(data[i][1])
      neg_gauss_val = bayes.gauss_val(val, neg_mean, neg_std_dev)
      pos_gauss_val = bayes.gauss_val(val, pos_mean, pos_std_dev)
      if neg_gauss_val > pos_gauss_val:
        predicted_class = 'NEG'
      else:
        predicted_class = 'POS'
      predictions.append([real_class, val, predicted_class])
    return predictions

  # returns the count of TP, TN, FP, FN for a given dataset
  def confusion_matrix(data):
    true_positives, true_negatives, false_positives, false_negatives = 0, 0, 0, 0
    for i in range(len(data)):
      if data[i][0] == 'POS' and data[i][2] == 'POS':
        true_positives +=1
      elif data[i][0] == 'NEG' and data[i][2] == 'NEG':
        true_negatives +=1
      elif data[i][0] == 'POS' and data[i][2] == 'NEG':
        false_negatives +=1
      elif data[i][0] == 'NEG' and data[i][2] == 'POS':
        false_positives +=1
    return true_positives, true_negatives, false_positives, false_negatives

  
  #following functions calculate various measures of performance
  def find_precision(TP, FP):
    return TP / (TP+FP)

  def find_sensitivity(TP, FN):
    return TP / (TP+FN)

  def find_specificity(TN, FP):
    return TN / (TN+FP)

  def find_accuracy(TP, TN, FP, FN):
    return (TP + TN) / (TP + TN + FP + FN)


  



In [5]:
# Get data from drive
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
data = []

#pick file to use
with open('/Users/jackmayr/PycharmProjects/my_juypter_notebooks/csc_395/data/BayesAssign1_01.csv') as csvfile:
	csvReader = csv.reader(csvfile)    
	for row in csvReader:        
		data.append(row)        
#print(data)
data_length = len(data)
train_percent = 0.7

# make our train and test sets
train, test = data[:int(data_length * train_percent)], data[int(data_length * train_percent):]

# split data by class
neg_data, pos_data = bayes.split_classes(train)

# find mean of each class
neg_mean = bayes.find_mean(neg_data)
pos_mean = bayes.find_mean(pos_data)

# find standard deviation of each class
neg_std_dev = bayes.find_std_dev(neg_data, neg_mean)
pos_std_dev = bayes.find_std_dev(pos_data, pos_mean)


pos_prevalence = bayes.find_prevalence(pos_data, train)

# Run predictive model on test set
test_predictions = bayes.find_predictions(test, neg_mean, neg_std_dev, pos_mean, pos_std_dev)

# Confusion Matrix quartiles
true_positives, true_negatives, false_positives, false_negatives = bayes.confusion_matrix(test_predictions)

# Performance stats
test_precision = bayes.find_precision(true_positives, false_positives)
test_sensitivity = bayes.find_sensitivity(true_positives, false_negatives)
test_specificity = bayes.find_specificity(true_negatives, false_positives)
test_accuracy = bayes.find_accuracy(true_positives, true_negatives, false_positives, false_negatives)

# printed outputs
print('Training Fraction: ', train_percent, data_length, len(train), len(test))
print('Estimated parameters (mean0, sig0, prob0, mean1,, sig1, prob1): ', round(neg_mean,2), round(neg_std_dev,2), round(1-pos_prevalence,2), round(pos_mean,2), round(pos_std_dev,2), round(pos_prevalence,2))
print('True POS: ', true_positives)
print('True NEG: ', true_negatives)
print('False POS: ', false_positives)
print('False NEG: ', false_negatives)
print('Prevalence: ', round(pos_prevalence,2))
print('Accuracy: ', round(test_accuracy,2))
print('Sensitivity: ', round(test_sensitivity,2))
print('Specificity: ', round(test_specificity,2))
print('Precsion: ', round(test_precision,2))
print('PPV: ', round(test_precision,2))

Training Fraction:  0.7 1000 700 300
Estimated parameters (mean0, sig0, prob0, mean1,, sig1, prob1):  0.05 1.04 0.77 3.03 0.95 0.23
True POS:  66
True NEG:  223
False POS:  9
False NEG:  2
Prevalence:  0.23
Accuracy:  0.96
Sensitivity:  0.97
Specificity:  0.96
Precsion:  0.88
PPV:  0.88


**Sample output from running the classification code.**

Training fraction: 0.7 10000 7000 3000

Estimated parameters (mean0, sig0, prob0, mean1, sig1, prob1): 0.0046 1.0103 0.4917 1.0268 2.0142 0.4917

True POS:  785

False POS: 217

True NEG:  1294

False NEG: 704


Prevalence:    0.496

Accuracy:      0.693

Sensitivity:   0.527

Specificity:   0.856

Precision:     0.783

PPV:           0.783


In [ ]:
'''
Histogram plotting - or other analysis students may like to do.

I used information from the links below to code histogram plots.

https://matplotlib.org/stable/gallery/statistics/hist.html
https://stackoverflow.com/questions/33203645/how-to-plot-a-histogram-using-matplotlib-in-python-with-a-list-of-data


'''